In [25]:
import array, csv, math, os, time
from datetime import timedelta, date, datetime


def FormatDateStr(date_str, format_str):
    return time.mktime(time.strptime(date_str, format_str))

def LngLatToWebMercator(lnglat):
    (lng, lat) = lnglat
    x = (lng + 180.0) * 256.0 / 360.0
    y = 128.0 - math.log(math.tan((lat + 90.0) * math.pi / 360.0)) * 128.0 / math.pi
    return [x, y]


def PackColor(color):
    return color[0] + color[1] * 256.0 + color[2] * 256.0 * 256.0;


In [3]:
raw_data = []
with open("../data/DL_FIRE_V1_36532/fire_archive_V1_36532.csv") as f:
    reader = csv.DictReader(f)
    for row in reader:
        raw_data.append(row)

In [11]:
len(raw_data)

20299549

In [4]:
raw_data[0]

{'acq_date': '2016-01-01',
 'acq_time': '2346',
 'bright_ti4': '333.1',
 'bright_ti5': '293.9',
 'confidence': 'n',
 'frp': '5.9',
 'instrument': 'VIIRS',
 'latitude': '19.40819',
 'longitude': '-155.27365',
 'satellite': 'N',
 'scan': '0.44',
 'track': '0.38',
 'type': '1',
 'version': '1'}

In [8]:
FormatDateStr(raw_data[0]['acq_date'], '%Y-%m-%d')

1451624400.0

In [10]:
LngLatToWebMercator([float(raw_data[0]['longitude']), float(raw_data[0]['latitude'])])

[17.58318222222222, 113.92685303861967]

In [30]:
#format x,y,packed_color,epoch_0,epoch_1
points = []
for row in raw_data:
    if row['confidence'] == 'h':
        x,y = LngLatToWebMercator([float(row['longitude']), float(row['latitude'])])
        packedColor = PackColor([185.0, 15.0, 7.0])
        epoch_0 = FormatDateStr(row['acq_date'], '%Y-%m-%d')
        epoch_1 = epoch_0 + 60*60*24*28
        points += [x,y,packedColor,epoch_0,epoch_1]

In [31]:
len(points)/5.0

1179157.0

In [35]:
oname = "../data/viirs-2016.bin"
array.array('f', points).tofile(open(oname, 'w'))
cmd = "scp %s timemachine2:/t/earthtime.org/app/data/viirs" % oname
!$cmd

viirs-2016.bin                                100%   22MB   8.2MB/s   00:02    
